# Downloading data

In [2]:
from Bio import Entrez
from Bio import SeqIO
import time
import os

Entrez.email = "brygida.serebriakova@gmail.com"

Using Entrez.esearch() to find current number of entries in NCBI and storing it in "ortho_count" variable.

In [2]:
ortho_search = Entrez.esearch(db = 'protein', term = 'txid11308[Organism:exp] NOT txid11320[Organism:exp] NOT txid11520[Organism:exp] NOT txid11552[Organism:exp] NOT txid1511084[Organism:exp] AND PB1')
ortho_record = Entrez.read(ortho_search)
ortho_count = int(ortho_record["Count"])
ortho_count

380

Using "ortho_count" as retmax parameter in Entrez.esearch() and extracting all IDs.

In [3]:
ortho = Entrez.esearch(db = 'protein', term = 'txid11308[Organism:exp] NOT txid11320[Organism:exp] NOT txid11520[Organism:exp] NOT txid11552[Organism:exp] NOT txid1511084[Organism:exp] AND PB1', retmax = ortho_count)
ortho_rec = Entrez.read(ortho)
ortho_ids = ortho_rec["IdList"]
len(ortho_ids)

380

Posting extracted IDs into Web environment and fetching them in batches of 100. 

In [14]:
#Web Environment. This parameter specifies the Web Environment that 
#contains the UID list to be provided as input to ESummary. Usually 
#this WebEnv value is obtained from the output of a previous ESearch, 
#EPost or ELink call. The WebEnv parameter must be used in 
#conjunction with query_key.

#Query key. This integer specifies which of the UID lists attached to the given 
#Web Environment will be used as input to ESummary.  Query keys are obtained
# from the output of previous ESearch, EPost or ELink calls.  The query_key 
#parameter must be used in conjunction with WebEnv.

In [4]:
ortho_epost_handle = Entrez.epost('protein', id = ','.join(ortho_ids))
ortho_epost_results = Entrez.read(ortho_epost_handle)

webenv, query_key = ortho_epost_results['WebEnv'], ortho_epost_results['QueryKey']

In [6]:
from io import StringIO

batchSize = 100
for start in range(0, ortho_count + batchSize, batchSize):
    try: 
        handle = Entrez.efetch(db = 'protein', rettype = 'fasta', retstart = start,
                              retmax = batchSize, webenv = webenv, query_key = query_key)
        
        ortho_PB1_fasta = list(SeqIO.parse(StringIO(handle.read()), 'fasta'))
        
        for i in range(len(ortho_PB1_fasta)):
            outputname = f"/Users/brigitaserebriakova/Desktop/Systems Biology/MAGISTRINIS/Orthomyxoviridae-PB1/{ortho_PB1_fasta[i].id}.fasta"
            SeqIO.write(ortho_PB1_fasta[i], outputname, 'fasta')    
    
    except:
        time.sleep(300)


# Filtering data

Removing all sequences that are shorter than 550 aa.

In [4]:
directory = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1"
destination = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Bin"

for filename in os.listdir(directory):
    
    f = os.path.join(directory, filename) 
    seq_record = SeqIO.read(f, 'fasta')
    
    if len(seq_record.seq) < 550:
        f_new = os.path.join(destination, filename)
        os.replace(f, f_new)

Manually adding one PB1 sequence from each group: Influenza A (infecting bats), Influenza B, Influenza C, Influenza D.

**GenBank IDs:**
* Influenza A: AKC43901.1
* Influenza B: AAU94857.1
* Influenza C: ALJ78713.1
* Influenza D: CEE50061.1

# Building PSSM profiles

Downloaded RdRp profiles (source: https://github.com/czbiohub/california-mosquito-study/blob/master/data/darkmatter/RdRp_profiles.zip).

Since only Orthomyxoviridae profile will be needed, all other profiles in directory are deleted.

Building PSSM profile (code source: https://github.com/czbiohub/california-mosquito-study/blob/master/scripts/PSSM_classification.ipynb)

## Supaprastinti kodą

In [9]:
%%bash

base_path=/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/RdRp_profiles ## where alignments are kept
cd $base_path

for msa in *full.fasta ## iterate over alignments of each Pfam group
    do
    echo $msa
    out=${msa/_full.fasta/_pssm.txt}
    hmmbuild $out $msa ## build PSSM
    
done;

cat *_pssm.txt > RdRp_profiles.pssm.txt ## concatenate Pfam PSSMs into a single file
hmmpress -f RdRp_profiles.pssm.txt ## compress for search

OrthomyxoRdRp_PF00602_full.fasta
# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             OrthomyxoRdRp_PF00602_full.fasta
# output HMM file:                  OrthomyxoRdRp_PF00602_pssm.txt
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     OrthomyxoRdRp_PF00602_full    59   878   716     1.88  0.590 

# CPU time: 0.53u 0.02s 00:00:00.55 Elapsed: 00:00:00.67
Working...    done.
Pressed and indexed 1 HMMs (1 names).
Models pressed into binary file:   RdRp_profiles.pssm.txt.h3m
SSI index for binary model file:   RdRp_profiles.pssm.txt.h3i
P

In [11]:
%%bash

cd /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS

cat ./Orthomyxoviridae-PB1/*.fasta >> Orthomyxo-PB1-candidates.fasta

Searching Orthomyxoviridae PB1 sequences against OrthomyxoRdRp profile (code source: https://github.com/czbiohub/california-mosquito-study/blob/master/scripts/PSSM_classification.ipynb).

In [13]:
%%bash

profiledb=/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/RdRp_profiles/RdRp_profiles.pssm.txt ## PSSMs of RdRps
candidates=/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-candidates.fasta ## proteins to be scanned with PSSMs

out=${candidates/.fasta/.pssm.out}

hmmscan --noali -o $out $profiledb $candidates ## scan away

In [29]:
from Bio.SearchIO import HmmerIO

base_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/"
hitfile = os.path.join(base_path,'Orthomyxo-PB1-candidates.pssm.out')
hits = HmmerIO.Hmmer3TextParser(open(hitfile,'r'))
PB1 = []

count = 0
for query in hits:
    count += 1
    for hit in query.hits:
        if hit.evalue < 1e-3:
            PB1.append(query.id)
        else: 
            print('not significant', query.id, hit.evalue)

print(count)
print(len(PB1))


322
322


Code below concatenates fasta files chosen from above (read more about that and why the whole step with building profiles is needed).

In [33]:
fasta_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1"
output_fasta = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-HHMER.fasta"

with open(output_fasta, "a") as output_file:
    
    for ID in PB1:
        fasta_file = os.path.join(fasta_path, f"{ID}.fasta")
        with open(fasta_file, "r") as f:
            content = f.read()
            output_file.write(content)

In [2]:
%%bash

cd /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS

cd-hit -i Orthomyxo-PB1-HHMER.fasta -o Orthomyxo-PB1-CD-HIT -c 0.8

Program: CD-HIT, V4.8.1, Apr 07 2021, 02:35:32
Command: cd-hit -i Orthomyxo-PB1-HHMER.fasta -o
         Orthomyxo-PB1-CD-HIT -c 0.8

Started: Tue Apr 11 15:42:40 2023
                            Output                              
----------------------------------------------------------------
total seq: 322
longest and shortest : 841 and 559
Total letters: 239832
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 840680
Max number of word counting entries: 90461561

comparing sequences from          0  to        322

      322  finished        107  clusters

Approximated maximum memory consumption: 76M
writing new database
writing clustering information
program completed !

Total CPU time 0.21


# BLAST

In [1]:
clustered_seq = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-CD-HIT.clstr"
fasta_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-HHMER.fasta"
output_fasta = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/OrthomyxoPB1-BLAST.fasta"

Extracting representative IDs from each cluster.

In [1]:
with open(clustered_seq, "r") as clusters:
    c = clusters.readlines()
rep_ID_lines = [e for e in c if e.endswith("*\n")]
rep_IDs = [e.split(", >")[1].split("...")[0] for e in rep_ID_lines]
rep_IDs
        

['QPN36968.1',
 'UQT02520.1',
 'QMP82367.1',
 'UVF62194.1',
 'QMP82195.1',
 'QMP82285.1',
 'QMP82388.1',
 'QMP82402.1',
 'QMP82403.1',
 'QMP82112.1',
 'QMP82168.1',
 'QMP82344.1',
 'QMP82362.1',
 'QMP82338.1',
 'QPZ88437.1',
 'UMO75733.1',
 'QED21504.1',
 'UPT53723.1',
 'AVR52572.1',
 'QMP82355.1',
 'AVR52567.1',
 'UUG74181.1',
 'UMO75734.1',
 'AJG39094.1',
 'QMP82147.1',
 'QPL15381.1',
 'QMP82248.1',
 'UMO75726.1',
 'QMP82204.1',
 'QPL15288.1',
 'QPL15340.1',
 'UMO75713.1',
 'UUG74233.1',
 'QPL15368.1',
 'UMO75705.1',
 'UMO75722.1',
 'QMP82272.1',
 'URQ09139.1',
 'QIJ70032.1',
 'QMP82229.1',
 'QWC36487.1',
 'UUG74184.1',
 'AJG39084.1',
 'UMO75717.1',
 'DAZ85673.1',
 'AJG39092.1',
 'AJG39088.1',
 'QDK54862.1',
 'QEM39320.1',
 'AJG39089.1',
 'QPZ88434.1',
 'AJG39086.1',
 'QKK82920.1',
 'QPL15369.1',
 'QPZ88431.1',
 'UUG74234.1',
 'ACY56282.1',
 'UMO75729.1',
 'ASR92124.1',
 'UUT42596.1',
 'QPN36942.1',
 'URY50684.1',
 'UUT42625.1',
 'AIY25030.1',
 'QOE76794.1',
 'AQU42764.1',
 'UMO75720

In [3]:
from Bio.Blast import NCBIWWW
help(NCBIWWW.qblast)

Help on function qblast in module Bio.Blast.NCBIWWW:

qblast(program, database, sequence, url_base='https://blast.ncbi.nlm.nih.gov/Blast.cgi', auto_format=None, composition_based_statistics=None, db_genetic_code=None, endpoints=None, entrez_query='(none)', expect=10.0, filter=None, gapcosts=None, genetic_code=None, hitlist_size=50, i_thresh=None, layout=None, lcase_mask=None, matrix_name=None, nucl_penalty=None, nucl_reward=None, other_advanced=None, perc_ident=None, phi_pattern=None, query_file=None, query_believe_defline=None, query_from=None, query_to=None, searchsp_eff=None, service=None, threshold=None, ungapped_alignment=None, word_size=None, short_query=None, alignments=500, alignment_view=None, descriptions=500, entrez_links_new_window=None, expect_low=None, expect_high=None, format_entrez_query=None, format_object=None, format_type='XML', ncbi_gi=None, results_file=None, show_overview=None, megablast=None, template_type=None, template_length=None, username='blast', password=No

BLASTing extracted IDs.

In [3]:
for i in rep_IDs:
    try:
        result_handle = NCBIWWW.qblast("blastp", "nr", sequence = i, entrez_query = "txid11308[ORGN] NOT (txid11320[ORGN] OR txid11520[ORGN] OR txid11552[ORGN] OR txid1511084[ORGN])")
        with open(f"BLAST_PB1/{i}.xml", "w") as out_handle:
            out_handle.write(result_handle.read())
        result_handle.close()
    except:
        time.sleep(5)

Searching through each BLAST files for PB1 sequences, that weren't downloaded during initial download (avoiding inconsistencies in data annotation).

List of downloaded PB1 sequences:

In [35]:
PB1_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1"

PB1_IDs = [i.split(".fasta")[0] for i in os.listdir(PB1_path)]
PB1_IDs

['ADR77506.1',
 'QGA69818.1',
 'UUG74184.1',
 'QDK54862.1',
 'ACG56671.1',
 'QTW97779.1',
 'AAF72693.1',
 'QRW42449.1',
 'QOQ34693.1',
 'QPZ88437.1',
 'QMP82112.1',
 'AAC25959.1',
 'AYR04798.1',
 'UQS93311.1',
 'UUT42633.1',
 'UUT42623.1',
 'QMP82229.1',
 'ACV71155.1',
 'ACY56282.1',
 'UYL95487.1',
 'ABG65760.1',
 'QRW42450.1',
 'QHR77126.1',
 'UBB38850.1',
 'ABG65756.1',
 'UDY81342.1',
 'QRW42591.1',
 'QPN36968.1',
 'UDY81330.1',
 'ASM94088.1',
 'QEM39320.1',
 'UUT42597.1',
 'QMP82367.1',
 'AKC43901.1',
 'AFN73049.1',
 'UUT42615.1',
 'QJQ28600.1',
 'UUT42605.1',
 'QJQ28610.1',
 'AAU94857.1',
 'QIJ70032.1',
 'AVM87619.1',
 'UQT02520.1',
 'AEC12828.1',
 'ABG81414.1',
 'QKK82921.1',
 'APG77897.1',
 'AAK48525.1',
 'QBQ64972.1',
 'ABG65757.1',
 'QRW42451.1',
 'QGA87318.1',
 'UPT53756.1',
 'ACG56669.1',
 'ABF68026.1',
 'CAO02362.1',
 'QMP82168.1',
 'ULY68741.1',
 'UUT42596.1',
 'QPL15288.1',
 'AHB34055.1',
 'ANM86269.1',
 'AED98371.1',
 'QKK82920.1',
 'BBD20265.1',
 'AJG39096.1',
 'UUT42604

Checking IDs for HSPs in each hit.

In [24]:
from Bio import SearchIO
blast_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1"

count = 0
for file in os.listdir(blast_path):
    if not file.startswith('.'):
        blast_result = os.path.join(blast_path, file)
        count += 1
        print(blast_result, count)
        blast_qresult = SearchIO.read(blast_result, "blast-xml")
        for hit in blast_qresult:
            if len(hit) > 1:
                print(hit.id, len(hit))
                for hsp in hit:
                    print(hsp.hit.id)
                    #print(hsp.evalue)
    #hits = [hit.id.split("|")[1] for hit in blast_qresult]
    

/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QOE76794.1.xml 1
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QWC36487.1.xml 2
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AAU94857.1.xml 3
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/URY50684.1.xml 4
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QPL15347.1.xml 5
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82362.1.xml 6
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/UMO75726.1.xml 7
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/UQT02520.1.xml 8
gb|AFN73049.1| 2
gb|AFN73049.1|
gb|AFN73049.1|
gb|ASR92124.1| 2
gb|ASR92124.1|
gb|ASR92124.1|
ref|YP_009996585.1| 2
ref|YP_009996585.1|
ref|YP_009996585.1|
gb|AXL67889.1| 2
gb|AXL67889.1|
gb|AXL67889.1|
ref|YP_009508043.1| 2
ref|YP_009508043.1|
ref|YP_009508043.1|
ref|YP_00998746

/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82272.1.xml 71
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QFR36189.1.xml 72
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AAL67962.1.xml 73
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/UUG74234.1.xml 74
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82112.1.xml 75
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82331.1.xml 76
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AVM87639.1.xml 77
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QJQ28580.1.xml 78
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82195.1.xml 79
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82168.1.xml 80
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AQM37674.1.xml 81

UQT02520.1 - encodes PB2, each hit has 2 HSPs.

Generate a list of PB1 protein sequences that weren't downloaded during the first try:

In [44]:
c1 = 0

PB1_to_download = []

for file in os.listdir(blast_path): #iterate through each BLAST file in directory
    if not file.startswith('.'): #omitting .DS_Store file
        blast_result = os.path.join(blast_path, file)
        c1 += 1
        print(blast_result, c1) #file directory and iteration count
        blast_qresult = SearchIO.read(blast_result, "blast-xml")
        hits = []
        #Galbut funkcija, kuri visa loopa duoda:
        for h in blast_qresult: #iterate through EACH HIT
            if len(h.hsps) > 1 and h[0].evalue <= 1e-3: #choose hits that have more than 1 HSPs and E-value <= 1e-3
                #print(len(h.hsps), h[0].hit.id.split("|")[1], h[0].evalue) #print number of HSPs, ID of first HSP, E-value of first HSP
                hits.append(h[0].hit.id.split("|")[1])
            elif len(h) == 1 and h[0].evalue <= 1e-3:
                #print(h.id.split("|")[1], h[0].evalue)
                hits.append(h.id.split("|")[1])
        #print(hits)
        
        c2 = 0
        for h in hits:
            if h not in PB1_IDs and h not in PB1_to_download:
                PB1_to_download.append(h)
                c2 += 1
        print("PB1 sequences to download: ", c2)

print(PB1_to_download, len(PB1_to_download))


/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QOE76794.1.xml 1
PB1 sequences to download:  1
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QWC36487.1.xml 2
PB1 sequences to download:  4
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AAU94857.1.xml 3
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/URY50684.1.xml 4
PB1 sequences to download:  3
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QPL15347.1.xml 5
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82362.1.xml 6
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/UMO75726.1.xml 7
PB1 sequences to download:  1
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/UQT02520.1.xml 8
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Syste

PB1 sequences to download:  1
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QJQ28580.1.xml 78
PB1 sequences to download:  2
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82195.1.xml 79
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QMP82168.1.xml 80
PB1 sequences to download:  1
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AQM37674.1.xml 81
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QPN36968.1.xml 82
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/QPZ88437.1.xml 83
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/UMO75729.1.xml 84
PB1 sequences to download:  0
/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1/AAC25959.1.xml 85
PB1 sequences to download:  0
/U

Downloading more PB1s

In [55]:
handle = Entrez.efetch(db = 'protein', id = ",".join(PB1_to_download), rettype = 'fasta')
records = list(SeqIO.parse(handle, 'fasta'))
#records
for i in range(len(PB1_to_download)):
    if len(records[i].seq) >= 550: #removing sequences that are shorter than 550 aa
        print(records[i].id)
        outputname = f"/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1/{records[i].id}.fasta"
        SeqIO.write(records[i], outputname, 'fasta')

DAZ91106.1
UYL95485.1
UYL95483.1
UYL95481.1
UYL95484.1
UYL95486.1


In [56]:
%%bash

cd /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS

cat ./Orthomyxoviridae-PB1/*.fasta >> Orthomyxo-PB1-updated.fasta

In [57]:
%%bash

profiledb=/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/RdRp_profiles/RdRp_profiles.pssm.txt ## PSSMs of RdRps
candidates=/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-updated.fasta ## proteins to be scanned with PSSMs

out=${candidates/.fasta/.pssm.out}

hmmscan --noali -o $out $profiledb $candidates ## scan away

In [58]:
from Bio.SearchIO import HmmerIO

base_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/"
hitfile = os.path.join(base_path,'Orthomyxo-PB1-updated.pssm.out')
hits = HmmerIO.Hmmer3TextParser(open(hitfile,'r'))
PB1 = []

count = 0
for query in hits:
    count += 1
    for hit in query.hits:
        if hit.evalue < 1e-3:
            PB1.append(query.id)
        else: 
            print('not significant', query.id, hit.evalue)

print(count)
print(len(PB1))

328
328


In [59]:
fasta_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1"
output_fasta = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxo-PB1-HHMER-updated.fasta"

with open(output_fasta, "a") as output_file:
    
    for ID in PB1:
        fasta_file = os.path.join(fasta_path, f"{ID}.fasta")
        with open(fasta_file, "r") as f:
            content = f.read()
            output_file.write(content)

In [60]:
%%bash

cd /Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS

cd-hit -i Orthomyxo-PB1-HHMER-updated.fasta -o Orthomyxo-PB1-CD-HIT-updated -c 0.8

Program: CD-HIT, V4.8.1, Apr 07 2021, 02:35:32
Command: cd-hit -i Orthomyxo-PB1-HHMER-updated.fasta -o
         Orthomyxo-PB1-CD-HIT-updated -c 0.8

Started: Wed Jun 21 20:23:03 2023
                            Output                              
----------------------------------------------------------------
total seq: 328
longest and shortest : 841 and 559
Total letters: 244456
Sequences have been sorted

Approximated minimal memory consumption:
Sequence        : 0M
Buffer          : 1 X 10M = 10M
Table           : 1 X 65M = 65M
Miscellaneous   : 0M
Total           : 76M

Table limit with the given memory limit:
Max number of representatives: 836786
Max number of word counting entries: 90460865

comparing sequences from          0  to        328

      328  finished        109  clusters

Approximated maximum memory consumption: 76M
writing new database
writing clustering information
program completed !

Total CPU time 0.24


## Script draft

In [62]:
from Bio import SearchIO
blast_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/tBLASTn-PB1"


for file in os.listdir(blast_path):
    if not file.startswith('.'):
        print(file)
        blast_result = os.path.join(blast_path, file)
        blast_qresult = SearchIO.read(blast_result, "blast-xml")
        for hit in blast_qresult:
            print(hit)


QMP82362.1.xml
UMO75726.1.xml
UQT02520.1.xml
AJG39094.1.xml
QMP82344.1.xml
QMP82338.1.xml
Query: QMP82338.1
       polymerase PB1 [Coleopteran orthomyxo-related virus OKIAV179]
  Hit: gi|1963875724|emb|LR990212.1| (10008955)
       Hylaea fasciaria genome assembly, chromosome: 20
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0     767.30     801          [6:805]      [8173829:8176217]
          1         0     642.88     755         [41:794]      [7770735:7772976]
Query: QMP82338.1
       polymerase PB1 [Coleopteran orthomyxo-related virus OKIAV179]
  Hit: gi|2211166997|emb|OW052077.1| (18175767)
       Phragmatobia fuliginosa genome assembly, chromosome: 20
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score   

Query: AVR52567.1
       PB1 [Photinus pyralis orthomyxo-like virus 1]
  Hit: gi|2381767832|emb|OX376305.1| (50306085)
       Cantharis rufa genome assembly, chromosome: 3
 HSPs: ----  --------  ---------  ------  ---------------  ---------------------
          #   E-value  Bit score    Span      Query range              Hit range
       ----  --------  ---------  ------  ---------------  ---------------------
          0         0     777.70     784          [3:785]    [19782078:19784430]
          1         0     593.19     594        [193:785]    [19793234:19795016]
          2         0     593.19     594        [193:785]    [19570058:19571840]
          3         0     199.13     193          [3:196]    [19792665:19793244]
          4         0     199.13     193          [3:196]    [19571830:19572409]
          5  5.1e-146     487.26     493        [285:776]    [22202809:22204288]
          6   5.8e-70     264.23     248        [526:774]      [7522758:7523499]
          7   1.1e

In [5]:
blast_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1"
blast_result = os.path.join(blast_path, "QFR36189.1.xml")

from Bio import SearchIO
blast_qresult = SearchIO.read(blast_result, "blast-xml")
blast_qresult

QueryResult(id='QFR36189.1', 50 hits)

In [7]:
print(blast_qresult)

Program: blastp (2.14.0+)
  Query: QFR36189.1 (559)
         polymerase sub-unit PB1 [Thailand tick thogotovirus]
 Target: nr
   Hits: ----  -----  ----------------------------------------------------------
            #  # HSP  ID + description
         ----  -----  ----------------------------------------------------------
            0      1  gb|QFR36189.1|  polymerase sub-unit PB1 [Thailand tick ...
            1      1  ref|YP_009553280.1|  polymerase basic subunit 1 [Oz virus]
            2      1  gb|UUT42607.1|  MAG: polymerase PB1 [Orthomyxoviridae sp.]
            3      1  gb|UUT42606.1|  MAG: polymerase PB1 [Orthomyxoviridae sp.]
            4      1  gb|UUT42595.1|  MAG: polymerase PB1 [Orthomyxoviridae sp.]
            5      1  gb|QOQ34668.1|  PB1 protein [Thogotovirus dhoriense]
            6      1  gb|UUT42632.1|  MAG: polymerase PB1 [Orthomyxoviridae sp.]
            7      1  gb|QPA18207.1|  PB1, partial [Batken virus]
            8      1  gb|UUT42631.1|  MAG: pol

In [9]:
print(blast_qresult.hit_keys)

['gb|QFR36189.1|', 'ref|YP_009553280.1|', 'gb|UUT42607.1|', 'gb|UUT42606.1|', 'gb|UUT42595.1|', 'gb|QOQ34668.1|', 'gb|UUT42632.1|', 'gb|QPA18207.1|', 'gb|UUT42631.1|', 'ref|YP_009352882.1|', 'gb|QBQ64972.1|', 'gb|QOQ34674.1|', 'gb|AMN92168.2|', 'gb|QCF29600.1|', 'gb|QCO69326.1|', 'gb|AHB34055.1|', 'gb|QKK82921.1|', 'gb|UYX69296.1|', 'gb|AED98371.1|', 'gb|UUT42593.1|', 'gb|UYL95482.1|', 'gb|AHB34061.1|', 'gb|QOQ34693.1|', 'gb|QOQ34700.1|', 'ref|YP_145794.1|', 'gb|QOQ34655.1|', 'gb|QOQ34686.1|', 'gb|UUT42626.1|', 'gb|UUT42609.1|', 'gb|UUT42594.1|', 'gb|UUT42628.1|', 'gb|UUT42627.1|', 'gb|UDY81360.1|', 'gb|UDY81366.1|', 'gb|UDY81348.1|', 'gb|UDY81336.1|', 'gb|UDY81354.1|', 'gb|ULY68735.1|', 'gb|ULY68741.1|', 'gb|QGA69818.1|', 'gb|ULY68747.1|', 'gb|UDY81318.1|', 'gb|UDY81312.1|', 'gb|UDY81330.1|', 'gb|UDL13962.1|', 'gb|UBB38850.1|', 'gb|QPL15289.1|', 'gb|APP91612.1|', 'gb|QPL15346.1|', 'gb|QQN90103.1|']


In [19]:
hits = [hit.id.split("|")[1] for hit in blast_qresult]
# for hit in blast_qresult:
#     hits.append(hit.id.split("|")[1])
hits

['QFR36189.1',
 'YP_009553280.1',
 'UUT42607.1',
 'UUT42606.1',
 'UUT42595.1',
 'QOQ34668.1',
 'UUT42632.1',
 'QPA18207.1',
 'UUT42631.1',
 'YP_009352882.1',
 'QBQ64972.1',
 'QOQ34674.1',
 'AMN92168.2',
 'QCF29600.1',
 'QCO69326.1',
 'AHB34055.1',
 'QKK82921.1',
 'UYX69296.1',
 'AED98371.1',
 'UUT42593.1',
 'UYL95482.1',
 'AHB34061.1',
 'QOQ34693.1',
 'QOQ34700.1',
 'YP_145794.1',
 'QOQ34655.1',
 'QOQ34686.1',
 'UUT42626.1',
 'UUT42609.1',
 'UUT42594.1',
 'UUT42628.1',
 'UUT42627.1',
 'UDY81360.1',
 'UDY81366.1',
 'UDY81348.1',
 'UDY81336.1',
 'UDY81354.1',
 'ULY68735.1',
 'ULY68741.1',
 'QGA69818.1',
 'ULY68747.1',
 'UDY81318.1',
 'UDY81312.1',
 'UDY81330.1',
 'UDL13962.1',
 'UBB38850.1',
 'QPL15289.1',
 'APP91612.1',
 'QPL15346.1',
 'QQN90103.1']

In [22]:
PB1_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/Orthomyxoviridae-PB1"

PB1_IDs = [i.split(".fasta")[0] for i in os.listdir(PB1_path)]
PB1_IDs

['ADR77506.1',
 'QGA69818.1',
 'UUG74184.1',
 'QDK54862.1',
 'ACG56671.1',
 'QTW97779.1',
 'AAF72693.1',
 'QRW42449.1',
 'QOQ34693.1',
 'QPZ88437.1',
 'QMP82112.1',
 'AAC25959.1',
 'AYR04798.1',
 'UQS93311.1',
 'UUT42633.1',
 'UUT42623.1',
 'QMP82229.1',
 'ACV71155.1',
 'ACY56282.1',
 'UYL95487.1',
 'ABG65760.1',
 'QRW42450.1',
 'QHR77126.1',
 'UBB38850.1',
 'ABG65756.1',
 'UDY81342.1',
 'QRW42591.1',
 'QPN36968.1',
 'UDY81330.1',
 'ASM94088.1',
 'QEM39320.1',
 'UUT42597.1',
 'QMP82367.1',
 'AKC43901.1',
 'AFN73049.1',
 'UUT42615.1',
 'QJQ28600.1',
 'UUT42605.1',
 'QJQ28610.1',
 'AAU94857.1',
 'QIJ70032.1',
 'AVM87619.1',
 'UQT02520.1',
 'AEC12828.1',
 'ABG81414.1',
 'QKK82921.1',
 'APG77897.1',
 'AAK48525.1',
 'QBQ64972.1',
 'ABG65757.1',
 'QRW42451.1',
 'QGA87318.1',
 'UPT53756.1',
 'ACG56669.1',
 'ABF68026.1',
 'CAO02362.1',
 'QMP82168.1',
 'ULY68741.1',
 'UUT42596.1',
 'QPL15288.1',
 'AHB34055.1',
 'ANM86269.1',
 'AED98371.1',
 'QKK82920.1',
 'BBD20265.1',
 'AJG39096.1',
 'UUT42604

In [26]:
count = 0
PB1_to_download = []
for hit in hits:
    if hit not in PB1_IDs:
        PB1_to_download.append(hit)
        count += 1
print("PB1 sequences to download: ", count)
print(PB1_to_download)
#print(count, len(hits), len(PB1_IDs))

QPL15289.1
PB1 sequences to download:  1
['QPL15289.1']


In [2]:
blast_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1"
blast_result = os.path.join(blast_path, "QFR36189.1.xml")

#from Bio.Blast import NCBIXML
from Bio.SearchIO import BlastIO
blast_records = BlastIO.BlastXmlParser(blast_result)

print(blast_records._ELEM_HIT)
#for i in blast_records.id:
#    print(i)

/Users/brigitaserebriakova/miniconda3/envs/orthomyxo/lib/python3.11/site-packages/Bio/SearchIO/_legacy/__init__.py:12: BiopythonDeprecationWarning: The 'Bio.SearchIO._legacy' module for parsing BLAST plain text output is deprecated and will be removed in a future release of Biopython. Consider generating your BLAST output for parsing as XML or tabular format instead.
  warnings.warn(


AttributeError: 'BlastXmlParser' object has no attribute '_ELEM_HIT'

In [3]:
blast_path = "/Users/brigitaserebriakova/Desktop/Systems-Biology/MAGISTRINIS/BLAST_PB1"
blast_result = open(os.path.join(blast_path, "UUG74235.1.xml"))

from Bio.Blast import NCBIXML
brec = NCBIXML.parse(blast_result)
brec

<generator object parse at 0x1124feb90>

In [4]:
blast_result = open("BLAST_results.xml")

from Bio.Blast import NCBIXML
blast_records = NCBIXML.parse(blast_result)
blast_records

FileNotFoundError: [Errno 2] No such file or directory: 'BLAST_results.xml'

In [27]:
for alignment in blast_records.alignments:
    print(alignment)

gb|QPN36968.1| hypothetical protein [Ganwon-do orthomyxo-like virus]
           Length = 841

gb|ASR92124.1| polymerase PB1 [Araguari virus]
           Length = 774

ref|YP_009110686.1| polymerase basic 1 protein [Wellfleet Bay virus] >gb|AIY25030.1| polymerase basic 1 protein [Wellfleet Bay virus]
           Length = 770

ref|YP_009987461.1| polymerase basic 1 protein [Lake Chad virus] >gb|QQO86214.1| polymerase basic 1 protein [Lake Chad virus]
           Length = 777

gb|UTQ48795.1| polymerase PB1 [Quaranjavirus quaranfilense]
           Length = 777

gb|AFN73049.1| polymerase PB1 [Tjuloc virus]
           Length = 777

gb|AXL67889.1| polymerase PB1 [Quaranjavirus quaranfilense]
           Length = 777

ref|YP_009508043.1| polymerase PB1 [Quaranjavirus quaranfilense] >sp|D0QX26.1| RecName: Full=RNA-directed RNA polymerase catalytic subunit [Quaranfil virus isolate Tick/Afghanistan/EGT377/1968] >gb|ACY56282.1| polymerase PB1 [Quaranjavirus quaranfilense]
           Length = 777

ref|

In [13]:
#with open(output_fasta, "a") as output_file:
    #for i in rep_IDs:
with open(fasta_path, "r") as f:
    content = f.readlines()

full_IDs = []
for i in rep_IDs:
    for e in content:
        if i in e:
            full_IDs.append(e)
full_IDs

['>QPN36968.1 hypothetical protein [Ganwon-do orthomyxo-like virus]\n',
 '>UQT02520.1 polymerase PB2, partial [Red mite quaranjavirus]\n',
 '>QMP82367.1 polymerase PB1 [Zygentoman orthomyxo-related virus OKIAV204]\n',
 '>UVF62194.1 MAG: PB1, partial [Bat faecal associated orthomyxo-like virus 1]\n',
 '>QMP82195.1 polymerase PB1, partial [Coleopteran orthomyxo-related virus OKIAV158]\n',
 '>QMP82285.1 polymerase PB1, partial [Siphonapteran orthomyxo-related virus OKIAV157]\n',
 '>QMP82388.1 polymerase PB1, partial [Lepidopteran orthomyxo-related virus OKIAV1731]\n',
 '>QMP82402.1 polymerase PB1, partial [Lepidopteran orthomyxo-related virus OKIAV178]\n',
 '>QMP82403.1 polymerase PB1, partial [Hemipteran orthomyxo-related virus OKIAV188]\n',
 '>QMP82112.1 polymerase PB1, partial [Hymenopteran orthomyxo-related virus OKIAV173]\n',
 '>QMP82168.1 polymerase PB1, partial [Coleopteran orthomyxo-related virus OKIAV200]\n',
 '>QMP82344.1 polymerase PB1, partial [Blattodean orthomyxo-related vir

In [14]:
for i in full_IDs:
    x = content.index(i)
    print(x)

2536
3473
2151
4205
1959
2062
2166
2181
2196
1914
1944
2106
2136
2091
2581
3374
1578
3431
1331
2121
1316
3531
3389
836
1929
2495
2032
3345
1988
2411
2426
3286
3618
2466
3271
3330
2047
3488
1701
2003
2981
3576
705
3301
1509
806
746
1563
1593
761
2566
732
1887
2481
2552
3633
532
3360
1194
3711
2522
3503
4085
691
2211
1062
3316
1426
2225
1276
2253
181
1208
2267
1248
2239
1262
891
1974
3066
3038
2018
2077
1901
0
1400
2441
2596
4260
13
971
168
1731
1181
625
3025
984
1290
1222
865
1235
4220
2454
2510
720
3647
1621


In [18]:
print(content[1959])
print(content[1959+1])

>QMP82195.1 polymerase PB1, partial [Coleopteran orthomyxo-related virus OKIAV158]

QQKQKQAEQQTTMSLENSFEFEEHYNSFLLNTLLDEEVAKNHFRHDFTKCTHNELLNNLS



In [19]:
content

['>AAA42968.1 polymerase [Thogotovirus dhoriense]\n',
 'MNLFSPRTGLSPTETQELLYAYTGPAPVAYGTRTRAVLENVLRPYKYFYKEENVPQALHI\n',
 'KTGQKGPEEISTTSPSSGFHRDSVILLSRELKAKYPEAFERLKAWIDCELLEMEYAELSK\n',
 'GRQTLSFLRNRNQPAPIALEETIEYLQQNLGRPIGQSMLSYLRAVMEVLAMPKTTFTYEV\n',
 'ATNLARFDFEEYDDGETGPLMMHFEKEKKKITITITQAELWEKTCTLGTMWKHLERGRLN\n',
 'RRTIATPSMLARGFVKIVEDAARVLLECLPSSGVPVGGEEKLAKLSSKLEAVSEVTGELS\n',
 'GDQEKFNECLDPDAMRLMWTVFLEDYPQWVKELFNIPFLIFKAKIADIGEGLTYQKEGVV\n',
 'RVFPFGEMPSEFDELLPNAIKDKEGKIVGIRCTLGMFMGMFNLSSTLLALIAADRSEITG\n',
 'DHVESSDDFIHFFKAKSYDDMFKQAELLRWSLKLVGINMSPSKCILISPAGIGEFNSKYH\n',
 'HRDFVGNVATDLPSLVPGGKNPSSDLAMGLNVIRHSINTNQMNFISGDLALRIFTKAYRH\n',
 'SYMAEGITRRTKFLEAFKKDPVLLNQGAPTVHSVSTLHLDEVCLRYQMHLLGEEELRRIM\n',
 'NPSNPITARTEEVVSFRPEGKLPMILEDNSVGSCFKYTFTRNRTVTDKPHRVLLEKEQQY\n',
 'QKITSFVEECFPELTIGNTTMPGTVKQACKRRLEYIIEQSDLPTEQKRALLEEMDS\n',
 '>AAC25959.1 PB1 protein [Thogotovirus thogotoense]\n',
 'MNLFTPRSEINPTTTQELLYAYTGPAPVAYGTRTRAVLENIIRPYQYFYKEPNVQRALDI\n',
 'KTGCKEPEDINVEGPSSGFH